# CourtRankRL FAISS Index Építés – Lokális/Cloud GPU

## Specifikáció
- **Input**: Pre-computed embeddings (`embeddings.npy`) és chunk ID mapping (`embedding_chunk_ids.json`)
- **Index**: OPQ64_256,IVF65536,PQ64x4fsr (optimal memory/accuracy balance)
- **Metrika**: Inner Product (IP) - L2-normalizált vektorokkal
- **Kimenet**: `faiss_index.bin` és `chunk_id_map.npy`

## Környezet-függő futtatás
- **Lokális (M3 MacBook Air)**: CPU-only training, memory-optimized
- **Cloud GPU (RunPod)**: GPU-accelerated training, faster build
- **Rugalmas path konfiguráció**: workspace vagy lokális artifacts

## FAISS paraméterek (agents.md szerint)
- **nlist**: 65536 (1M-10M vektorokhoz)
- **nprobe**: 256-1024 (recall optimalizálás)
- **Train sample**: 1,966,080 vektor (adaptív)
- **Memory footprint**: ~2-3GB (M3 MacBook Air kompatibilis)


In [ ]:
%pip install --upgrade --quiet faiss-gpu tqdm numpy psutil


⚠️ faiss-gpu nem elérhető, faiss-cpu telepítése...
❌ FAISS telepítés sikertelen
✅ Egyéb csomagok telepítve


/Users/zelenyianszkimate/Documents/CourtRankRL/.venv/bin/python: No module named pip
/Users/zelenyianszkimate/Documents/CourtRankRL/.venv/bin/python: No module named pip
/Users/zelenyianszkimate/Documents/CourtRankRL/.venv/bin/python: No module named pip
/Users/zelenyianszkimate/Documents/CourtRankRL/.venv/bin/python: No module named pip
/Users/zelenyianszkimate/Documents/CourtRankRL/.venv/bin/python: No module named pip


In [2]:
import json
import os
import time
from pathlib import Path
from typing import List

import faiss
import numpy as np
import psutil
import torch
from tqdm import tqdm


In [ ]:
# --- Konfiguráció ---

# Rugalmas path konfiguráció - workspace vagy lokális artifacts
BASE_PATH = Path(os.getenv("WORKSPACE_PATH", "/workspace"))
ARTIFACTS_PATH = Path(os.getenv("ARTIFACTS_PATH", str(BASE_PATH)))

# Input fájlok
EMBEDDINGS_PATH = ARTIFACTS_PATH / "embeddings.npy"
CHUNK_IDS_PATH = ARTIFACTS_PATH / "embedding_chunk_ids.json"

# Output fájlok
FAISS_PATH = BASE_PATH / "faiss_index.bin"
CHUNK_MAP_PATH = BASE_PATH / "chunk_id_map.npy"

# FAISS paraméterek (agents.md szerint) - CPU-optimalizált alternatívák
EMBED_DIM = 768

# CPU-optimalizált nlist kalkuláció (dokumentáció alapján)
# N = vektorok száma, optimális nlist: 4*sqrt(N) - 16*sqrt(N)
N_ESTIMATED = 2_000_000  # becsült vektorok száma
NLIST_MIN = int(4 * (N_ESTIMATED ** 0.5))   # ~5656
NLIST_MAX = int(16 * (N_ESTIMATED ** 0.5))  # ~22624
NLIST_TARGET = min(16384, NLIST_MAX)  # CPU-ra csökkentve 65536-ról

# Alternatív index konfigurációk CPU-hoz
INDEX_ALTERNATIVES = [
    f"OPQ64_256,IVF{NLIST_TARGET},PQ64x4fsr",  # jelenlegi (csökkentett nlist)
    f"IVF{NLIST_TARGET},PQ64x4fs,RFlat",       # PQ + RFlat újraranking
    f"HNSW32",                                 # HNSW alternatíva
    f"OPQ64,IVF{NLIST_TARGET//4},PQ64x4fsr",   # negyed nlist
]

NPROBE_TARGET = min(256, NLIST_TARGET // 4)  # recall optimalizálás
TRAIN_SAMPLE_SIZE = 1_966_080  # adaptív train sample
RNG_SEED = 42

np.random.seed(RNG_SEED)

print(f"📂 Base path: {BASE_PATH}")
print(f"📦 Artifacts path: {ARTIFACTS_PATH}")
print(f"📄 Embeddings: {EMBEDDINGS_PATH}")
print(f"📄 Chunk IDs: {CHUNK_IDS_PATH}")

# Ellenőrzés
if not EMBEDDINGS_PATH.exists():
    raise FileNotFoundError(f"❌ Nem található: {EMBEDDINGS_PATH}")
if not CHUNK_IDS_PATH.exists():
    raise FileNotFoundError(f"❌ Nem található: {CHUNK_IDS_PATH}")

BASE_PATH.mkdir(parents=True, exist_ok=True)
print("✅ Konfiguráció betöltve - FAISS INDEX ÉPÍTÉS MÓD")


📂 Base path: /workspace
📦 Artifacts path: /workspace
📄 Embeddings: /workspace/embeddings.npy
📄 Chunk IDs: /workspace/embedding_chunk_ids.json


FileNotFoundError: ❌ Nem található: /workspace/embeddings.npy

In [ ]:
# --- Embedding betöltés és robusztus validáció ---

print("="*80)
print("📥 EMBEDDING BETÖLTÉS")
print("="*80)

load_start = time.time()

# Embedding vektorok betöltése
print("🔄 Embedding vektorok betöltése...")
all_embeddings = np.load(EMBEDDINGS_PATH)
print(f"✅ Embeddings betöltve: {all_embeddings.shape}")

# Chunk ID lista betöltése
print("🔄 Chunk ID lista betöltése...")
with CHUNK_IDS_PATH.open("r", encoding="utf-8") as handle:
    all_chunk_ids = json.load(handle)
print(f"✅ Chunk IDs betöltve: {len(all_chunk_ids)}")

# Alap validáció (méret és dim)
n_vectors, d = all_embeddings.shape
assert d == EMBED_DIM, f"Embedding dim vártan {EMBED_DIM}, de {d}"
assert len(all_chunk_ids) == n_vectors, (
    f"Chunk ID-k száma ({len(all_chunk_ids)}) nem egyezik a vektorok számával ({n_vectors})"
)

# Robusztus tisztítás: szűrés NaN/Inf és zéró-normájú vektorokra
print("🧹 Embedding tisztítás (NaN/Inf/zéró normák)...")
finite_mask = np.isfinite(all_embeddings).all(axis=1)
norms = np.linalg.norm(all_embeddings, axis=1)
nonzero_mask = norms > 0
valid_mask = finite_mask & nonzero_mask

num_invalid = int((~valid_mask).sum())
if num_invalid > 0:
    print(f"⚠️ {num_invalid:,} problémás vektor kiszűrve (NaN/Inf vagy zéró-norma)")
    all_embeddings = all_embeddings[valid_mask]
    all_chunk_ids = [cid for cid, keep in zip(all_chunk_ids, valid_mask) if keep]
else:
    print("✅ Nincs NaN/Inf/zéró-norma hiba az embeddingekben")

# Biztonságos L2-normalizálás (clip a nullával osztás ellen)
norms = np.linalg.norm(all_embeddings, axis=1, keepdims=True)
all_embeddings = all_embeddings / np.clip(norms, 1e-12, None)

# Float32 konverzió
if all_embeddings.dtype != np.float32:
    all_embeddings = all_embeddings.astype(np.float32, copy=False)

# Végső ellenőrzések
assert np.isfinite(all_embeddings).all(), "Embedding mátrixban NaN/Inf maradt a tisztítás után"
final_norms = np.linalg.norm(all_embeddings, axis=1)
if not np.allclose(final_norms, 1.0, atol=1e-6):
    print("ℹ️ Normalizáció utáni normák nem pontosan 1 – újranormalizálás")
    all_embeddings = all_embeddings / np.clip(
        np.linalg.norm(all_embeddings, axis=1, keepdims=True), 1e-12, None
    )

# Metrikák frissítése a szűrés után
n_vectors, d = all_embeddings.shape

load_time = time.time() - load_start
print(f"⏱️ Betöltési idő: {load_time:.2f} másodperc")
print(f"📊 Összesített adatok: {n_vectors:,} vektor, {d} dimenzió")
print(f"💾 Memória használat: {all_embeddings.nbytes / 1024**3:.2f} GB")


In [ ]:
# --- FAISS Index Építés ---

print("="*80)
print("🏗️ FAISS INDEX ÉPÍTÉS")
print("="*80)

build_start = time.time()

# 1) CPU-optimalizált index kiválasztás
print(f"📊 Dataset: {n_vectors:,} vektor, {d} dimenzió")
print(f"📏 Optimális nlist tartomány: {NLIST_MIN:,} - {NLIST_MAX:,}")

# CPU-barát index kiválasztás próbálgatással
def select_cpu_friendly_index():
    """CPU-n működő index kiválasztása próbálgatással"""
    for i, factory_str in enumerate(INDEX_ALTERNATIVES):
        print(f"🔍 Próbálkozás {i+1}/{len(INDEX_ALTERNATIVES)}: {factory_str}")
        try:
            index = faiss.index_factory(d, factory_str, faiss.METRIC_INNER_PRODUCT)
            print(f"✅ Sikeres: {factory_str}")
            return index, factory_str
        except Exception as e:
            print(f"❌ Sikertelen: {factory_str} - {type(e).__name__}: {e}")
            continue
    raise RuntimeError("❌ Egyik index sem működik CPU-n")

index_cpu, factory_str = select_cpu_friendly_index()
nlist = getattr(index_cpu, 'nlist', NLIST_TARGET)  # HNSW-nál nincs nlist
print(f"✅ Végső index: {factory_str} (nlist: {nlist:,})")

# 2) Train minta kiválasztás
target_train = min(n_vectors, TRAIN_SAMPLE_SIZE)
rng = np.random.default_rng(RNG_SEED)
if n_vectors > target_train:
    print(f"🎲 Random train sample: {target_train:,} vektor")
    train_idx = rng.choice(n_vectors, size=target_train, replace=False)
    train_matrix = all_embeddings[train_idx]
else:
    print(f"📊 Összes vektor tréninghez: {n_vectors:,}")
    train_matrix = all_embeddings

train_matrix = np.ascontiguousarray(train_matrix.astype("float32", copy=False))

# 3) Device detection és training
USE_GPU = torch.cuda.is_available()
print(f"🚀 GPU elérhető: {USE_GPU}")

try:
    if USE_GPU:
        print("🚀 GPU-first tréning...")
        try:
            # Dinamikus import GPU funkciókhoz
            if hasattr(faiss, 'StandardGpuResources'):
                res = faiss.StandardGpuResources()
                index = faiss.index_cpu_to_gpu(res, 0, index_cpu)
                print("🎓 FAISS tréning indul (GPU)...")
                index.train(train_matrix)
                print("✅ GPU tréning kész")
            else:
                raise AttributeError("GPU funkciók nem elérhetőek")
        except (AttributeError, RuntimeError):
            print("⚠️ GPU funkciók nem elérhetőek (faiss-cpu), CPU training...")
            raise Exception("GPU not available")
    else:
        raise Exception("No GPU available")
except Exception as e:
    print(f"⚠️ GPU tréning sikertelen ({type(e).__name__}): {e}")
    print("🧠 CPU tréning (low-mem paramokkal)...")
    try:
        faiss.omp_set_num_threads(min(os.cpu_count() or 8, 16))
        print(f"🔧 OMP szálak: {faiss.omp_get_max_threads()}")
    except Exception as e2:
        print(f"⚠️ OMP konfiguráció sikertelen: {e2}")
    print("🎓 FAISS tréning indul (CPU)...")
    index = index_cpu
    index.train(train_matrix)
    print("✅ CPU tréning kész")

# 5) nprobe beállítása
index.nprobe = min(int(NPROBE_TARGET), nlist)
print(f"🎯 nprobe: {index.nprobe}")

# 6) Vektorok hozzáadása batch-ben
def add_in_batches(faiss_index, vectors, batch_size=200_000):
    """Batch-es hozzáadás memória optimalizálással"""
    N = vectors.shape[0]
    added = 0
    for start in range(0, N, batch_size):
        end = min(N, start + batch_size)
        faiss_index.add(vectors[start:end])
        added = end
        if (start // batch_size) % 5 == 0 or end == N:
            print(f"➕ Add progress: {added:,}/{N:,}")
    return added

print("📥 Vektorok hozzáadása az indexhez (batch)...")
added = add_in_batches(index, all_embeddings, batch_size=200_000)
print(f"✅ Index méret: {index.ntotal:,} vektor (added: {added:,})")

build_time = time.time() - build_start
print(f"⏱️ Build idő: {build_time:.2f} másodperc")


In [ ]:
# --- Mentés és validáció ---

print("="*80)
print("💾 MENTÉS ÉS VALIDÁCIÓ")
print("="*80)

save_start = time.time()

# 7) GPU → CPU konverzió mentéshez (ha szükséges)
if isinstance(index, faiss.GpuIndex):
    print("🔁 GPU → CPU konverzió mentéshez...")
    index_cpu_final = faiss.index_gpu_to_cpu(index)
else:
    index_cpu_final = index

# 8) FAISS index mentése
print("💾 FAISS index mentése...")
faiss.write_index(index_cpu_final, str(FAISS_PATH))
print(f"✅ Mentve: {FAISS_PATH}")

# 9) Chunk ID mapping mentése (npy formátumban)
print("💾 Chunk ID mapping mentése (npy)...")
chunk_ids_array = np.asarray(all_chunk_ids, dtype=object)
np.save(CHUNK_MAP_PATH, chunk_ids_array)
print(f"✅ Mentve: {CHUNK_MAP_PATH} (shape={chunk_ids_array.shape})")

save_time = time.time() - save_start
total_time = time.time() - load_start

# 10) Teljesítmény metrikák
mem_usage = psutil.virtual_memory().used / 1024**3
idx_size_gb = FAISS_PATH.stat().st_size / 1024**3

print("="*80)
print("📊 TELJESÍTMÉNY METRIKÁK")
print("="*80)
print(f"⏱️ Teljes feldolgozási idő: {total_time:.2f} másodperc")
print(f"⏱️ Betöltési idő: {load_time:.2f} másodperc")
print(f"⏱️ Build idő: {build_time:.2f} másodperc")
print(f"⏱️ Mentési idő: {save_time:.2f} másodperc")
print(f"💾 Memória használat: {mem_usage:.1f}GB")
print(f"📦 Index fájlméret: {idx_size_gb:.2f}GB")
print(f"🧮 Vektorok: {index_cpu_final.ntotal:,}")
print(f"📏 nlist: {nlist:,}, nprobe: {index.nprobe}")
print(f"🏗️ Index típusa: {factory_str}")

print("🎉 FAISS index építése sikeres!")
print(f"   Index: {FAISS_PATH.name}")
print(f"   Mapping: {CHUNK_MAP_PATH.name}")
print(f"   Vektorok: {index_cpu_final.ntotal:,}")
print(f"   Index típusa: {factory_str}")
print(f"   nlist: {nlist:,}, nprobe: {index.nprobe}")


## Összefoglaló
- A FAISS dense index sikeresen elkészült CPU-optimalizált konfigurációval.
- L2-normalizált vektorokkal Inner Product metrika használatával.
- Kimeneti fájlok: `faiss_index.bin` és `chunk_id_map.npy` a megadott könyvtárban.
- A notebook automatikusan kiválasztja a CPU-n működő legjobb indexet.

### CPU-optimalizált index kiválasztás:
A rendszer próbálgatással választja ki a CPU-n működő legjobb indexet:
1. **OPQ64_256,IVF16384,PQ64x4fsr** (csökkentett nlist)
2. **IVF16384,PQ64x4fs,RFlat** (PQ + újraranking)
3. **HNSW32** (HNSW alternatíva)
4. **OPQ64,IVF4096,PQ64x4fsr** (negyed nlist)

### Környezet-függő optimalizálások:
- **Lokális (M3)**: CPU-only training, memory-optimized, OMP thread limiting
- **Cloud GPU**: GPU-accelerated training, faster build times (ha elérhető)
- **Rugalmas path**: workspace vagy lokális artifacts használata

### FAISS paraméterek (dinamikus):
- **nlist**: 5656-22624 tartomány (4*sqrt(N)-16*sqrt(N) alapján)
- **nprobe**: nlist/4 (recall optimalizálás)
- **Train sample**: 1,966,080 vektor (adaptív)
- **Memory footprint**: ~2-3GB (M3 MacBook Air kompatibilis)

### Használat:
```bash
# Lokális futtatás (M3 MacBook Air):
export ARTIFACTS_PATH="/path/to/artifacts"
jupyter notebook faiss_index_builder.ipynb

# Cloud futtatás (RunPod):
export WORKSPACE_PATH="/workspace"
jupyter notebook faiss_index_builder.ipynb
```

### Következő lépés:
A `scripts/hybrid_retrieval.py` script használja ezt az indexet a hybrid retrieval-hez.
